In [1]:
import re
import os
import json
import statistics
import datetime
import warnings
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df = pl.read_parquet(os.path.join("data","cnb_vyber.parquet"))
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('pocet_stran'))

In [15]:
kafka = df.filter(pl.col("100_7") == "jn19990218037").with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
kafka.group_by("rok").len().sort("rok")

rok,len
datetime[μs],u32
1928-01-01 00:00:00,1
1929-01-01 00:00:00,2
1931-01-01 00:00:00,1
1935-01-01 00:00:00,1
1946-01-01 00:00:00,1
…,…
2020-01-01 00:00:00,2
2021-01-01 00:00:00,1
2022-01-01 00:00:00,4


In [47]:
alt.Chart(kafka.group_by("rok").len().sort("rok").to_pandas()).mark_bar().encode(alt.X("rok:T"),alt.Y('len:Q'))

alt.Chart(...)

In [49]:
alt.Chart(df.filter(pl.col("100_7") == 'jk01040240').group_by('rok').len().sort('rok').with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime)).to_pandas()).mark_bar().encode(alt.X("rok:T"),alt.Y('len:Q'))

alt.Chart(...)

In [55]:
df.filter(pl.col("100_a").str.contains('Škvore'))

008,020_a,020_c,020_q,041_a,041_h,041_ind1,044_a,072_a,072_x,080_a,100_4,100_7,100_a,100_d,240_l,245_a,245_c,245_n,245_p,246_a,250_a,250_b,260_a,260_b,260_c,260_e,260_f,260_ind1,264_a,264_b,264_c,264_ind2,300_a,300_b,300_c,490_a,490_v,500_a,521_a,546_a,648_a,650_a,650_x,650_y,650_z,651_a,653_a,655_a,700_4,700_7,700_a,700_d,710_4,710_7,710_a,710_b,830_a,928_a,964_a,leader,001,rok,pocet_stran
str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64,i64
"""980119s1963 xr g 0…",null,"[""Kčs 6,00""]","[""(Vázáno) :""]",null,null,null,null,"[""821.162.3-3""]","[""Česká próza""]","[""821.162.3-32"", ""(0:82-32)""]","[""aut""]","""jk01130413""","""Škvorecký, Josef,""","""1924-2012""",null,"""Legenda Emöke /""","""Josef Škvorecký""",null,null,null,"[""Vydání 1.""]",null,null,null,null,null,null,null,"[""Praha :""]","[""Československý spisovatel,""]","[""1963""]","[""1""]","[""78 stran ;""]",null,"[""18 cm""]",null,null,"[""V roce 1963 dotisk"", ""10000 výtisků""]",null,null,null,null,null,null,null,null,null,"[""české novely"", ""Czech novellas""]",null,null,null,null,null,null,null,null,null,null,null,""" nam a22 i 4500""","""bk196303619""",1963,78
"""970915s1964 xr 0…",null,null,"[""(Váz.)""]",null,null,null,null,null,null,"[""885-321.2-32:940.5(=924:437)""]","[""aut""]","""jk01130413""","""Škvorecký, Josef,""","""1924-2012""",null,"""Sedmiramenný svícen /""","""Josef Škvorecký ; il. Františe…",null,null,null,"[""1. vyd.""]",null,"[""Praha :""]","[""Naše vojsko,""]","[""1964""]",null,null,null,null,null,null,null,"[""129, [2] s. :""]","[""il. ;""]","[""21 cm""]","[""Živé knihy ;""]","[""sv. 53""]","[""Obálka, vazba a tit. dvoulist: ilustrátor"", ""20000 výt.""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""kn20020322091""]","[""Svaz protifašistických bojovníků""]",null,"[""Živé knihy (Naše vojsko)""]",null,"[""Židé - válka světová 1939-1945 - povídky"", ""Válka světová 1939-1945 - židé - povídky""]",""" nam a22 1 4500""","""bk196400765""",1964,129
"""970903s1965 xr a 0…",null,null,"[""(Váz.)""]",null,null,null,null,null,null,"[""885-321.2-32:940.5(=924:437)""]","[""aut""]","""jk01130413""","""Škvorecký, Josef,""","""1924-2012""",null,"""Sedmiramenný svícen /""","""Josef Škvorecký ; [ilustrace F…",null,null,null,"[""2. vyd.""]",null,"[""Praha :""]","[""Naše vojsko,""]","[""1965""]",null,null,null,null,null,null,null,"[""129, [2] s. :""]","[""il. ;""]","[""8°""]","[""Živé knihy : edice Svazu protifašistických bojovníků ;""]","[""sv. 53""]","[""35000 výt.""]",null,null,null,null,null,null,null,null,null,null,"[""ill""]","[""jk01043203""]","[""Hudeček, František,""]","[""1909-1990""]",null,null,null,null,"[""Živé knihy (Naše vojsko)""]",null,"[""Válka světová 1939-1945 - židé - povídky"", ""Židé - válka světová 1939-1945 - povídky""]",""" nam a22 1a 4500""","""bk196502081""",1965,129
"""971021s1965 xr …",null,null,null,null,null,null,null,null,null,"[""885-321.1""]","[""aut""]","""jk01130413""","""Škvorecký, Josef,""","""1924-2012""",null,"""Legenda Emöke /""","""Josef Škvorecký""",null,null,null,"[""2. vyd.""]",null,"[""Praha :""]","[""Československý spisovatel,""]","[""1965""]","[""(Přerov :""]","[""Tisk 04)""]",null,null,null,null,null,"[""71, [2] s. ;""]",null,"[""8°""]","[""Život kolem nás. Malá řada ;""]","[""Sv. 15""]","[""Obálku s použitím kresby Marca Chagalla a vazbu navrhl Zdenek Seydl"", ""10000 výt.""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[

In [53]:
alt.Chart(df.filter(pl.col("100_7") == 'jk01070894').group_by('rok').len().sort('rok').with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime)).to_pandas()).mark_bar().encode(alt.X("rok:T"),alt.Y('len:Q'))

alt.Chart(...)

In [57]:
alt.Chart(df.filter(pl.col("100_7") == 'jk01130413').group_by('rok').len().sort('rok').with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime)).to_pandas()).mark_bar().encode(alt.X("rok:T"),alt.Y('len:Q'))

alt.Chart(...)